In [2]:
import pandas as pd

In [4]:
df1 = pd.read_csv('fear_greed_index.csv')
df2 = pd.read_csv('historical_data.csv')

In [6]:
df1.head()

,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


In [35]:
df2.sample(5)

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
21896,0x4f93fead39b70a1824f981a54d4e55b278e9f760,ETH,3064.800000,12.4456,38143.27,SELL,03-05-2024 21:11,5.466060e+01,Close Long,822.529704,0x7619f78ab65263d41c9e0408bd9077020256005a20fc...,21381700218,True,13.350146,1.010000e+13,1.710000e+12
191814,0xbee1707d6b44d4d52bfe19e41f8a828645437aab,HYPE,17.028000,118.4200,2016.46,BUY,18-04-2025 05:52,-4.172070e+03,Close Short,18.745886,0x8fa9ddf2c3fa775a5c080421c01e2f02011800564f97...,87255757332,True,0.604936,1.480000e+14,1.740000e+12
152586,0x2c229d22b100a7beb69122eed721cee9b24011dd,kBONK,0.057042,12590.0000,718.16,SELL,20-11-2024 08:27,9.634520e+05,Close Long,18.608020,0xeed057feee01114a81c304173c2cd601b9007d605926...,48644423205,True,0.251355,9.280000e+14,1.730000e+12
181656,0xbee1707d6b44d4d52bfe19e41f8a828645437aab,HYPE,15.294000,32.6900,499.96,BUY,20-03-2025 06:15,2.335000e+01,Open Long,0.000000,0x0405cddc165a66d36372041fe3ee50015f00944b9284...,80854375005,True,0.149988,3.720000e+14,1.740000e+12
119038,0x8170715b3b381dffb7062c0298972d4727a0a63b,MELANIA,0.520000,292.6000,152.15,BUY,23-04-2025 22:05,-1.076358e+06,Close Short,10.513118,0xfb61410d549d7ca0cc3304221b515002038900ac147a...,88368093091,False,0.014606,6.430000e+14,1.750000e+12


In [12]:
df1['classification'].value_counts()

classification
Fear             781
Greed            633
Extreme Fear     508
Neutral          396
Extreme Greed    326
Name: count, dtype: int64

In [25]:
df1.columns

Index(['timestamp', 'value', 'classification', 'date'], dtype='object')

In [27]:
df2.columns

Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp'],
      dtype='object')

In [37]:

df1['date'] = pd.to_datetime(df1['date'])
df2['Timestamp IST'] = pd.to_datetime(df2['Timestamp IST'], format='%d-%m-%Y %H:%M')
df2['date'] = df2['Timestamp IST'].dt.normalize()

merged_df = pd.merge(df2, df1, on='date', how='left')

print('Merged DataFrame Info:')
merged_df.info()
print('\nMerged DataFrame Head:')
print(merged_df.head())
print('\nMissing values in merged_df:')
print(merged_df.isnull().sum())


Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211224 entries, 0 to 211223
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Account           211224 non-null  object        
 1   Coin              211224 non-null  object        
 2   Execution Price   211224 non-null  float64       
 3   Size Tokens       211224 non-null  float64       
 4   Size USD          211224 non-null  float64       
 5   Side              211224 non-null  object        
 6   Timestamp IST     211224 non-null  datetime64[ns]
 7   Start Position    211224 non-null  float64       
 8   Direction         211224 non-null  object        
 9   Closed PnL        211224 non-null  float64       
 10  Transaction Hash  211224 non-null  object        
 11  Order ID          211224 non-null  int64         
 12  Crossed           211224 non-null  bool          
 13  Fee               211224 non-null  f

In [39]:
merged_df.to_csv("merged_data.csv", index=False)

In [49]:

merged_df['date'] = pd.to_datetime(merged_df['date'])

# Calculate daily average PnL
daily_pnl = merged_df.groupby('date')['Closed PnL'].sum().reset_index()

# Merge with sentiment data
sentiment_pnl = pd.merge(daily_pnl, merged_df[['date', 'value', 'classification']].drop_duplicates(), on='date', how='inner')

# Calculate correlation between sentiment value and daily PnL
correlation = sentiment_pnl['value'].corr(sentiment_pnl['Closed PnL'])
print(f"Correlation between sentiment value and daily Closed PnL: {correlation}")

# Analyze average PnL by sentiment classification
avg_pnl_by_sentiment = sentiment_pnl.groupby('classification')['Closed PnL'].mean().sort_values(ascending=False)
print("\nAverage Closed PnL by Sentiment Classification:")
print(avg_pnl_by_sentiment)




Correlation between sentiment value and daily Closed PnL: -0.08264200129021725

Average Closed PnL by Sentiment Classification:
classification
Extreme Fear     52793.589178
Fear             36891.818040
Extreme Greed    23817.292199
Neutral          19297.323516
Greed            11140.566181
Name: Closed PnL, dtype: float64


In [51]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate daily average PnL
daily_pnl = merged_df.groupby("date")["Closed PnL"].sum().reset_index()

# Merge with sentiment data
sentiment_pnl = pd.merge(daily_pnl, merged_df[["date", "value", "classification"]].drop_duplicates(), on="date", how="inner")

# Plotting sentiment value vs daily Closed PnL
plt.figure(figsize=(10, 6))
sns.scatterplot(x="value", y="Closed PnL", data=sentiment_pnl)
plt.title("Sentiment Value vs. Daily Closed PnL")
plt.xlabel("Sentiment Value (Fear to Greed)")
plt.ylabel("Daily Closed PnL")
plt.grid(True)
plt.tight_layout()
plt.savefig("sentiment_vs_pnl_scatterplot.png")
plt.close()

# Plotting average PnL by sentiment classification
avg_pnl_by_sentiment = sentiment_pnl.groupby("classification")["Closed PnL"].mean().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=avg_pnl_by_sentiment.index, y=avg_pnl_by_sentiment.values)
plt.title("Average Closed PnL by Sentiment Classification")
plt.xlabel("Sentiment Classification")
plt.ylabel("Average Closed PnL")
plt.tight_layout()
plt.savefig("avg_pnl_by_sentiment_barplot.png")
plt.close()

In [53]:
# Define trader performance metrics
trader_performance = merged_df.groupby("Account").agg(
    total_closed_pnl=("Closed PnL", "sum"),
    num_trades=("Order ID", "count"),
    avg_closed_pnl_per_trade=("Closed PnL", "mean")
).reset_index()

# Segment traders based on total_closed_pnl (example segmentation)
def segment_trader(pnl):
    if pnl > 10000:
        return "Highly Profitable"
    elif pnl > 0:
        return "Profitable"
    elif pnl < -10000:
        return "Highly Unprofitable"
    else:
        return "Unprofitable"

trader_performance["pnl_segment"] = trader_performance["total_closed_pnl"].apply(segment_trader)

print("Trader Performance Metrics and Segmentation:")
print(trader_performance.head())
print("\nDistribution of Trader Segments:")
print(trader_performance["pnl_segment"].value_counts())

# Analyze characteristics of different trader segments (e.g., average num_trades, avg_closed_pnl_per_trade)
print("\nAverage Metrics by PnL Segment:")
print(trader_performance.groupby("pnl_segment").agg(
    avg_num_trades=("num_trades", "mean"),
    avg_avg_closed_pnl_per_trade=("avg_closed_pnl_per_trade", "mean")
))


Trader Performance Metrics and Segmentation:
                                      Account  total_closed_pnl  num_trades  \
0  0x083384f897ee0f19899168e3b1bec365f52a9012      1.600230e+06        3818   
1  0x23e7a7f8d14b550961925fbfdaa92f5d195ba5bd      4.788532e+04        7280   
2  0x271b280974205ca63b716753467d5a371de622ab     -7.043619e+04        3809   
3  0x28736f43f1e871e6aa8b1148d38d4994275d72c4      1.324648e+05       13311   
4  0x2c229d22b100a7beb69122eed721cee9b24011dd      1.686580e+05        3239   

   avg_closed_pnl_per_trade          pnl_segment  
0                419.127768    Highly Profitable  
1                  6.577654    Highly Profitable  
2                -18.492043  Highly Unprofitable  
3                  9.951530    Highly Profitable  
4                 52.071011    Highly Profitable  

Distribution of Trader Segments:
pnl_segment
Highly Profitable      29
Highly Unprofitable     3
Name: count, dtype: int64

Average Metrics by PnL Segment:
                 

In [57]:
# Convert 'date' column to datetime if not already
merged_df["date"] = pd.to_datetime(merged_df["date"])

# Define trader performance metrics
trader_performance = merged_df.groupby("Account").agg(
    total_closed_pnl=("Closed PnL", "sum"),
    num_trades=("Order ID", "count"),
    avg_closed_pnl_per_trade=("Closed PnL", "mean")
).reset_index()

# Segment traders based on total_closed_pnl (example segmentation)
def segment_trader(pnl):
    if pnl > 10000:
        return "Highly Profitable"
    elif pnl > 0:
        return "Profitable"
    elif pnl < -10000:
        return "Highly Unprofitable"
    else:
        return "Unprofitable"

trader_performance["pnl_segment"] = trader_performance["total_closed_pnl"].apply(segment_trader)

# Merge trader segments back to the main dataframe
merged_df = pd.merge(merged_df, trader_performance[["Account", "pnl_segment"]], on="Account", how="left")

# Analyze trading behavior by segment and sentiment
segment_sentiment_analysis = merged_df.groupby(["pnl_segment", "classification"]).agg(
    avg_pnl=("Closed PnL", "mean"),
    total_trades=("Order ID", "count"),
    avg_trade_size_usd=("Size USD", "mean")
).reset_index()

print("Trading Behavior by Trader Segment and Sentiment Classification:")
print(segment_sentiment_analysis)

Trading Behavior by Trader Segment and Sentiment Classification:
           pnl_segment classification     avg_pnl  total_trades  \
0    Highly Profitable   Extreme Fear   37.989084         20181   
1    Highly Profitable  Extreme Greed   68.000450         39728   
2    Highly Profitable           Fear   54.792435         57968   
3    Highly Profitable          Greed   54.444738         48008   
4    Highly Profitable        Neutral   35.049780         36108   
5  Highly Unprofitable   Extreme Fear  -22.598398          1219   
6  Highly Unprofitable  Extreme Greed   51.702381           264   
7  Highly Unprofitable           Fear   46.768567          3869   
8  Highly Unprofitable          Greed -202.027754          2295   
9  Highly Unprofitable        Neutral   17.327765          1578   

   avg_trade_size_usd  
0         5369.165361  
1         3112.406288  
2         8027.263141  
3         5728.862967  
4         4799.801341  
5         5028.002691  
6         3088.967992  
7    